In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from models.reader import data_reader
import config
from utils.data_utils import dump_pkl, write_vocab, load_pkl, build_vocab, load_vocab

In [ ]:
col_sep=config.col_sep
data_path=config.train_seg_path
min_count=config.min_count

In [ ]:
print(data_path)

In [ ]:
data_content, data_lbl = data_reader(data_path, col_sep)
word_lst = []
for i in data_content:
    word_lst.extend(i.split())

# word vocab
word_vocab = build_vocab(word_lst, min_count=min_count, sort=True, lower=True)

In [ ]:
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 2),vocabulary=word_vocab, sublinear_tf=True)


In [ ]:
print(vectorizer)

In [ ]:
len(data_content)

In [ ]:
data_feature = vectorizer.fit_transform(data_content)

In [ ]:
print(vectorizer)

In [ ]:
data_feature.shape

In [ ]:
type(data_feature)


In [ ]:
arr = data_feature.toarray()

In [ ]:
arr[1]

In [ ]:
count = 0
for ele in data_feature:
    if count == 2:
        print(ele.shape)
        print(ele)
        break
    count += 1

In [ ]:
import numpy as np

In [ ]:
vectorizer_count = CountVectorizer(analyzer='word', ngram_range=(1, 2),vocabulary=word_vocab)


In [ ]:
features = vectorizer_count.fit_transform(data_content[:10])

In [ ]:
arr = features.toarray()

In [ ]:
arr.shape

In [ ]:
import pickle as pkl
import codecs

In [ ]:
with codecs.open("/ssd1/zhubenchang/output/feature_tf_word.pkl", "rb") as f:
    vect = pkl.load(f)

In [ ]:
res = vect.fit_transform(["我 在 北京", "你 在 哪里"])

In [ ]:
res = vect.fit_transform(data_content[:10000])

In [ ]:
len(vect.get_feature_names())

In [ ]:
vect.get_feature_names()[:5]

In [ ]:
len(vect.vocabulary_)

In [ ]:
with codecs.open("/ssd1/zhubenchang/output/model_tf_word_logistic_regression.pkl", "rb") as f:
    lr_model = pkl.load(f)

In [ ]:
lr_model.predict([[0]*114319, [1]*114319])

In [ ]:
arr = lr_model.predict(res)

In [ ]:
data_lbl = np.array(data_lbl)
data_content = np.array(data_content)


In [ ]:
len(data_lbl)
len(data_content)

In [ ]:
data_lbl[:10]

In [ ]:
data_content_yl = data_content[data_lbl=="1104"]

In [ ]:
len(data_content_yl)

In [ ]:
len(data_content)

In [ ]:
context = "济南 老年公寓 首选 济南 舜 康   多元化 综合性 的 养老 机构 !   济南 舜 康 老年公寓 是 历城区 首家 老年公寓   专业 养老 机构   家人 更 放心   ! 我们 有 合理 的 医疗 护理 方案 给 老人   给 老人 提供 最 贴心 的 服务   欢迎来电 咨询 !   济南 老年公寓 哪家 好 ? 济南 舜 康   设置 完善   服务 更 贴心   济南 舜 康 老年公寓 成立 于 2007 年   老牌 老年公寓 更 值得信赖   设施完善   专业 护理 .   尊重 老人 的 个性化 需求   提高 老年人 的 晚年生活 质量   家人 放心 的 选择 !   济南 舜 康 老年公寓   专业 护理 服务   家人 放心 的 选择 !   济南 舜 康 老年公寓   是 集 养老   托 老   医疗   康复   及 临终关怀 为 一体 的 多元化   综合性 的 养老 机构 ! 给 老人 提供 贴心 的 服务   家人 安心   老人 舒心 的 选择 !   济南 舜 康 老年公寓   专业 护理 服务   家人 放心 的 选择 !   济南 舜 康 老年公寓   是 集 养老   托 老   医疗   康复   及 临终关怀"

In [ ]:
res = vect.fit_transform(data_content_yl)
pre_res = lr_model.predict(res)

In [ ]:
data_content_yl[pre_res!=16]

In [ ]:
weights = lr_model.coef_

In [ ]:
weight = weights[16]

In [ ]:
sorted(zip(vect.get_feature_names(), weight), key = lambda x:x[1], reverse = True)

In [ ]:
arr[arr!=0]

In [ ]:
np.arange(1,10)

In [ ]:
import sys
sys.getsizeof(vect)/1024**2

In [ ]:
len(data_content)

In [ ]:
sys.getsizeof(lr_model)

In [ ]:
from keras.layers import Dense, Dropout, Flatten, Input, MaxPooling1D, Convolution1D
from keras.layers import Embedding
from keras.layers import GlobalAveragePooling1D
from keras.layers import LSTM, Bidirectional, TimeDistributed
from keras.layers.merge import Concatenate
from keras.models import Model, load_model
from keras.models import Sequential

In [ ]:
cnn = load_model("/ssd1/zhubenchang/output/model_vectorize_cnn.pkl")

In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_content)


In [ ]:
sequence = tokenizer.texts_to_sequences(data_content)

In [ ]:
import sys
import pickle as pkl
import codecs 

In [ ]:
sys.getsizeof(tokenizer)

In [ ]:
with codecs.open("./output/cnn_tokenizer","wb") as f:
    pkl.dump(tokenizer, f)

In [ ]:
with codecs.open("./output/cnn_tokenizer","rb") as f:
    token = pkl.load(f)

In [ ]:
print(token)

In [ ]:
tok_res = token.texts_to_sequences(data_content[:2])

In [ ]:
len(tok_res[0])

In [ ]:
len(tok_res[1])

In [ ]:
pad_res = pad_sequences(tok_res, maxlen = 200)

In [ ]:
pad_res.shape

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
sequence_pad = pad_sequences(sequence, maxlen = 200)

In [ ]:
from keras.utils import to_categorical

In [ ]:
label_id = load_vocab(config.label_vocab_path)

In [ ]:
label_id

In [ ]:
data_label = [label_id[i] for i in data_lbl]

In [ ]:
lbl = to_categorical(data_label, num_classes = 26)

In [ ]:
lbl.shape

In [ ]:
lbl[1]

In [ ]:
sequence_pad[0]

In [ ]:
for layer in cnn.layers:
    print(layer.get_config())
    print(layer.__class__.__name__)

In [ ]:
res = cnn.predict(sequence_pad[:1])

In [ ]:
res

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

In [ ]:
cnn.evaluate(sequence_pad[:100000],lbl[:100000])

In [ ]:
res.max()

In [ ]:
res

In [ ]:
res == res.max()

In [ ]:
np.where(res == res.max())

In [ ]:
a = [1,2,3]
a.index(2)

In [ ]:
cnn.predict()